# Ejercicio de Feedback 1
---

## Apartado 1: Ejercicio de Deep Learning
Implementar una red neuronal profunda (Deep Neural Network) para la clasificación de
imágenes en un conjunto de datos de reconocimiento de objetos. Se espera que los estudiantes
diseñen, entrenen y evalúen la red neuronal utilizando técnicas de aprendizaje profundo, como
la selección de arquitectura, la optimización de hiperparámetros y la visualización de resultados.
Pasos Clave:
1. Preparación de datos: Cargar y preprocesar el conjunto de datos.
2. Diseño de la arquitectura: Seleccionar una arquitectura de red neuronal profunda
adecuada.
3. Entrenamiento del modelo: Entrenar la red neuronal utilizando los datos de
entrenamiento.
4. Evaluación del modelo: Evaluar el rendimiento del modelo en el conjunto de prueba.
5. Visualización de resultados: Visualizar ejemplos de imágenes y las predicciones
realizadas por la red neuronal.

In [16]:
# Lets see if CUDA is available
import tensorflow as tf

if tf.test.is_built_with_cuda():
    print("TensorFlow was built with CUDA")
else:
    print("TensorFlow was not built with CUDA")

print("GPUs available: ", tf.config.list_physical_devices('GPU'))


TensorFlow was built with CUDA
GPUs available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 1. Preparacion de Datos Cargar y preprocesar el conjunto de datos
---

El conjunto de datos escogido es el fashion mnist, escogido principalmente para poder descargarlo directamente de la librería Keras, permitiendo al correctos poder usar y ejecutar

In [17]:
import tensorflow_datasets as tfds
import pandas as pd
import os
# Descargamos eldataset, generamos un dataframe y lo guardamos como pickle
# para no tener que volverlo a descargar la siguiente ejecución
# Dado que los labels son numericos, descargaremos también la info
# para poder saber que raza significa ese numero
train_dataset, info_train = tfds.load('stanford_dogs', split='train', with_info=True, as_supervised=True)
test_dataset, info_test = tfds.load('stanford_dogs', split='test', with_info=True, as_supervised=True)




In [18]:
# Preprocesamiento de las imagenes
import tensorflow as tf

# Usaremos imagenes de 64x64 para reducir el numero
# de neuronas necesario y por tanto mejorar el rendimiento
# es cierto, que perderá información y será menos preciso por ello

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
])

size_image = [100,100]
def image_transformation_train(image, label):
  # usaremos 2^7 pixeles
  image = tf.image.resize(image, size_image)
  # es cierto que el color es las razas de perros ayuda a distinguir, pero para
  # mejorar el rendimiento demomento usaremos la escala de grises
  image = tf.image.rgb_to_grayscale(image)
  # normalizar la imagen en 255
  image = image/255
  image = data_augmentation(image)

  return image, label

def image_transformation_test(image, label):
  # usaremos 2^7 pixeles
  image = tf.image.resize(image, size_image)
  # es cierto que el color es las razas de perros ayuda a distinguir, pero para
  # mejorar el rendimiento demomento usaremos la escala de grises
  image = tf.image.rgb_to_grayscale(image)
  # normalizar la imagen en 255
  image = image/255

  return image, label
train_dataset = train_dataset.shuffle(1000)

train_dataset = train_dataset.map(image_transformation_train)
test_dataset = test_dataset.map(image_transformation_test)
train_dataset = train_dataset.batch(32)
test_dataset = test_dataset.batch(32)

##  2.Diseño de la arquitectura
---
Tras el preprocesado generaremos el modelo al que le podemos indicar el numero de clases con el fin de hacerlo genérico para cualquier dataset

In [23]:
import tensorflow.keras.models as models
from tensorflow.keras import regularizers

def create_model(num_clases, num_dense=120, activ_conv='relu',
                 activ_dense='softmax', padding='valid', l2_value=0.01):
  return tf.keras.models.Sequential([
    ## con el fin de captar mejor las formas de las imagenes, utilizaremos capas
    # de convolución, seguidas de Max Pooling para reducir la dimensionalidad
    # entre capas.
    # primera capa conv-pool
    tf.keras.layers.Conv2D(filters=100, kernel_size=(5, 5),
                           activation=activ_conv, input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2), padding=padding),
    # segunda capa conv-pool
    tf.keras.layers.Conv2D(filters=150, kernel_size=(5, 5),
                           activation=activ_conv),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2)),
    tf.keras.layers.Conv2D(filters=200, kernel_size=(5, 5),
                           activation=activ_conv),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2)),
    # usamos flatten para preparar el output de antes para la capa Dense, ya que
    # estas esperan una entrada unidimensional.
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(num_dense, activation='relu',
                          kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)),
    tf.keras.layers.Dense(num_dense, activation='relu',
                          kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)),
    #ultima capa, deberia tener el mismo numero de neuronas que de clases
    tf.keras.layers.Dense(num_clases, activation='softmax')
])



## 3. Entrenamiento del modelo
---
En esta fase generaremos varios modelos con diferentes hiperparámetros puestos
con el fin de comparar sus precisiones finales

In [ ]:
##Generar los modelos
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD

#model_1 = create_model(120, num_dense=200, activ_conv='relu',
#                 activ_dense='relu', padding='same')
file_path = 'best_model_1.keras'
#if os.path.isfile(file_path):
#  print("Cargando modelo")
#  model_1 = tf.keras.models.load_model(file_path)
#else:
print("Creando modelo")
model_1 = create_model(120, num_dense=512, activ_conv='relu',
                  activ_dense='softmax', padding='same', l2_value=0.0001)
print(model_1.summary())
## optimizer, usamos descendiente del gradiente
# al aumentar o disminuir momentum dependeremos mas o menos de los gradientes
# calculados anteriormente para el proximo salto (actualizacion de pesos)
optimizer_a = SGD(learning_rate=0.01, momentum=0.9)

# ya use el AdmaW optimizer, pero me funciona peor para este caso que SGD

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    verbose=1,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,
    patience=2,
    verbose=1,
    min_lr=0.000001
)

model_1.compile(
    optimizer=optimizer_a,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint_1 = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max' )

## Entrenar los modelos, se guarda el mejor resultado hasta el momento
num_epochs = 40
batch_size = 32
history_1 = model_1.fit(train_dataset, epochs=num_epochs, batch_size=batch_size,
                        verbose=1, validation_data=(test_dataset),
                        callbacks=[checkpoint_1, early_stopping, reduce_lr])

Creando modelo
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 96, 96, 100)       2600      
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 48, 48, 100)       0         
 ng2D)                                                           
                                                                 
 conv2d_18 (Conv2D)          (None, 44, 44, 150)       375150    
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 22, 22, 150)       0         
 ng2D)                                                           
                                                                 
 conv2d_19 (Conv2D)          (None, 18, 18, 200)       750200    
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 9,